# Makeover Monday: May

Author: Hasnain Hossain

This analysis is performed using Python Jupyter Notebook, Pandas, and Plotly for visualizations. For each of the figures, gover over for more details and utilize the select or deselect on legend to explore the visualizations further. 

<b>Jump to [Analytic-Questions](#Analytic-Questions) for synopsis of analysis.<b>

# Detailed Analysis

## Package Imports

In [185]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
import numpy as np
import math
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-cites-trade-script")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 300)
px.set_mapbox_access_token(open(".mapbox_token").read())
mapbox_access_token = open(".mapbox_token").read()

## Import CITES Data

In [186]:
cites_trade_df = pd.read_csv('./MayMakeoverMonday.csv')

In [187]:
cites_trade_df

,Year,App.,Taxon,Class,Order,Family,Genus,Importer,Exporter,Origin,Importer reported quantity,Exporter reported quantity,Term,Unit,Purpose,Source
0,2019,I,Araucaria araucana,NaN,Pinales,Araucariaceae,Araucaria,NL,CL,NaN,91300.00,NaN,live,NaN,T,D
1,2019,I,Balaenoptera acutorostrata,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,JP,BR,NaN,278.13,NaN,bones,g,S,W
2,2019,I,Balaenoptera acutorostrata,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,JP,BR,NaN,130.33,NaN,specimens,g,S,W
3,2019,I,Balaenoptera bonaerensis,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,JP,BR,NaN,25.05,NaN,bones,g,S,W
4,2019,I,Balaenoptera bonaerensis,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,JP,BR,NaN,4.52,NaN,specimens,g,S,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,2020,III,Potamotrygon motoro,Elasmobranchii,Myliobatiformes,Potamotrygonidae,Potamotrygon,CA,CO,NaN,NaN,12.0,live,NaN,T,W
936,2020,III,Potamotrygon motoro,Elasmobranchii,Myliobatiformes,Potamotrygonidae,Potamotrygon,IL,CO,NaN,NaN,4.0,live,NaN,T,W
937,2020,III,Potamotrygon motoro,Elasmobranchii,Myliobatiformes,Potamotrygonidae,Potamotrygon,MX,CO,NaN,NaN,4.0,live,NaN,T,W
938,2020,III,Potamotrygon orbignyi,Elasmobranchii,Myliobatiformes,Potamotrygonidae,Potamotrygon,MX,CO,NaN,NaN,4.0,live,NaN,T,W


In [188]:
# create dataframe where import and export quantity is reported.
cites_trade_import_df = cites_trade_df[(cites_trade_df['Importer reported quantity'].notna())]
cites_trade_export_df = cites_trade_df[(cites_trade_df['Exporter reported quantity'].notna())]

In [189]:
cites_trade_both_df = cites_trade_df[(cites_trade_df['Exporter reported quantity'].notna()) & (cites_trade_df['Importer reported quantity'].notna())]

## Import Country Data

In [204]:
plotly_countries_and_continents = pd.read_csv('./continents2.csv')
country_code = pd.read_csv('./country_code.csv')

In [205]:
def getLatLon(country, rtr_var):
    """
    gets the lat or lon for a given country
    """
    try:
        loc = geolocator.geocode(country['country_name'])
        lat = loc.latitude
        lon = loc.longitude
    except:
        try:
            loc = geolocator.geocode(country['alpha_3'])
            lat = loc.latitude
            lon = loc.longitude
        except:
            try:
                loc = geolocator.geocode(country['alpha_2'])
                lat = loc.latitude
                lon = loc.longitude
            except:
                lat = 'na'
                lon = 'na'
    if (rtr_var == 'lat'):
        return lat
    else:
        return lon

## What is the most important trade in terms of volume? The least?

### Data Transformation

In [206]:
cites_trade_impt_df = pd.DataFrame(cites_trade_df.groupby(['Taxon','Importer'], as_index=False)['Importer reported quantity'].sum())
cites_trade_impt_df = cites_trade_impt_df.rename(columns={'Importer reported quantity': 'Quantity', 'Importer':'Country'})
cites_trade_impt_df['Type'] = 'Import'

In [207]:
cites_trade_expt_df = pd.DataFrame(cites_trade_df.groupby(['Taxon','Exporter'], as_index=False)['Exporter reported quantity'].sum())
cites_trade_expt_df = cites_trade_expt_df.rename(columns={'Exporter reported quantity': 'Quantity', 'Exporter':'Country'})
cites_trade_expt_df['Type'] = 'Export'

In [208]:
cites_trade_vol_df = cites_trade_impt_df.append(cites_trade_expt_df)
cites_trade_vol_df = cites_trade_vol_df[cites_trade_vol_df['Quantity'] != 0]
cites_trade_vol_df = cites_trade_vol_df.sort_values("Quantity", ascending=False).reset_index(drop=True)

In [209]:
summary_stats = cites_trade_vol_df.agg(
    {
       "Quantity": ["min", "max", "median", "mean", "std"],
    }
)
mean_one_stdv = summary_stats['Quantity']['mean'] + (summary_stats['Quantity']["std"])

In [210]:
cites_trade_vol_most_df = cites_trade_vol_df[cites_trade_vol_df['Quantity']>=mean_one_stdv]
cites_trade_vol_least_df = cites_trade_vol_df[cites_trade_vol_df['Quantity']<mean_one_stdv]

### The most important trade in terms of volume

In [211]:
fig_most = px.histogram(cites_trade_vol_most_df, 
            x=cites_trade_vol_most_df.Taxon, 
            y=cites_trade_vol_most_df.Quantity, 
            color=cites_trade_vol_most_df.Type, 
            hover_data=['Taxon', 'Type', 'Quantity'], 
            labels={
            "Type": "Type of Trade", 
            "Quantity": "Trade Quantity"
            },
            title="Most Important Trade in Terms of Volume")
fig_most.show()

### The least important trade in terms of volume

In [212]:
fig_least = px.histogram(cites_trade_vol_least_df, 
            x=cites_trade_vol_least_df.Taxon, 
            y=cites_trade_vol_least_df.Quantity, 
            color=cites_trade_vol_least_df.Type, 
            hover_data=['Taxon', 'Type', 'Quantity'], 
            labels={
            "Type": "Type of Trade", 
            "Quantity": "Trade Quantity"
            },
            title="Least Important Trade In Terms of Volume"
                        )
fig_least.show()

## What patterns exist between importing and exporting countries?

In [213]:
cites_trade_vol_iso = cites_trade_vol_df.merge(country_code, how='left', left_on=['Country'], right_on=['alpha_2'])

In [214]:
cites_trade_vol_iso['Quantity_log'] = cites_trade_vol_iso['Quantity'].apply(lambda x: abs(math.log(x)))

In [215]:
cites_trade_vol_iso

,Taxon,Country,Quantity,Type,country_name,alpha_2,alpha_3,lat,lon,Quantity_log
0,Cycas revoluta,HN,765779.000,Export,Honduras,HN,HND,15.257243,-86.075514,13.548649
1,Caiman crocodilus fuscus,CO,360929.000,Export,Colombia,CO,COL,2.889443,-73.783892,12.796437
2,Euphorbia antisyphilitica,JP,307766.470,Import,Japan,JP,JPN,36.574844,139.239418,12.637097
3,Strombus gigas,BS,265245.840,Export,Bahamas,BS,BHS,24.773655,-78.000055,12.488412
4,Euphorbia antisyphilitica,FR,170000.000,Import,France,FR,FRA,46.603354,1.888334,12.043554
...,...,...,...,...,...,...,...,...,...,...
573,Ovis canadensis,ZA,1.000,Import,South Africa,ZA,ZAF,-28.816624,24.991639,0.000000
574,Dalbergia frutescens,ES,0.120,Import,Spain,ES,ESP,39.326068,-4.837979,2.120264
575,Madracis auretenra,CO,0.082,Export,Colombia,CO,COL,2.889443,-73.783892,2.501036
576,Paubrasilia echinata,DE,0.036,Import,Germany,DE,DEU,51.083420,10.423447,3.324236


### Patterns of importing and exporting countries

In [216]:
mapbox_fig = px.scatter_mapbox(cites_trade_vol_iso, 
                        lat=cites_trade_vol_iso.lat, 
                        lon=cites_trade_vol_iso.lon,     
                        color=cites_trade_vol_iso.Type, 
                        size=cites_trade_vol_iso.Quantity_log,
                        size_max=20,
                        zoom=1, 
                        hover_data={'lat': False, 
                                    'lon': False, 
                                    'Quantity_log' : False, 
                                    'Taxon': True,
                                    'country_name' : True, 
                                    'Quantity' : True, 
                                    'Type': True}, 
                        labels={
                        "country_name": "Country Name",
                        "Type": "Type of Trade", 
                        "Quantity": "Trade Quantity"
                        },
                        title="Patterns of Importing & Exporting Countries")
mapbox_fig.show()

## What country/countries trade the largest number of endangered species and what country do they export to?

In [217]:
cites_trade_iso_df = cites_trade_df.merge(country_code, how='left', left_on=['Importer'], right_on=['alpha_2'])
cites_trade_iso_df = cites_trade_iso_df.rename(columns={'alpha_2':'Import_alpha_2',
                                            'alpha_3':'Import_alpha_3', 
                                            'country_name':'Import_country_name',
                                            'lat':'impt_lat',
                                            'lon':'impt_lon'})

In [218]:
cites_trade_iso_df = cites_trade_iso_df.merge(country_code, how='left', left_on=['Exporter'], right_on=['alpha_2'])
cites_trade_iso_df = cites_trade_iso_df.rename(columns={'alpha_2':'Export_alpha_2',
                                            'alpha_3':'Export_alpha_3',
                                            'country_name':'Export_country_name',
                                           'lat':'expt_lat',
                                           'lon':'expt_lon'})

In [219]:
cites_trade_impt_df = pd.DataFrame(cites_trade_iso_df.groupby(
    ['Importer','Exporter'], as_index=False).agg({'Importer reported quantity': 'sum', 'Taxon':'count'}))
cites_trade_impt_df = cites_trade_impt_df.rename(columns={'Importer reported quantity': 'Quantity', 'Taxon':'Coun_of_Taxon'})
cites_trade_impt_df['Type'] = 'Import'

In [220]:
cites_trade_expt_df = pd.DataFrame(cites_trade_iso_df.groupby(
    ['Importer','Exporter'], as_index=False).agg({'Exporter reported quantity': 'sum', 'Taxon':'count'}))
cites_trade_expt_df = cites_trade_expt_df.rename(columns={'Exporter reported quantity': 'Quantity', 'Taxon':'Coun_of_Taxon'})
cites_trade_expt_df['Type'] = 'Export'

In [221]:
cites_trade_count_df = cites_trade_impt_df.append(cites_trade_expt_df)
cites_trade_count_df = cites_trade_count_df[cites_trade_count_df['Quantity'] != 0]

In [222]:
cites_trade_count_df = cites_trade_count_df.sort_values("Coun_of_Taxon", ascending=False).reset_index(drop=True)

In [223]:
cites_trade_count_iso_df = cites_trade_count_df.merge(country_code, how='left', left_on=['Exporter'], right_on=['alpha_2'])
cites_trade_count_iso_df = cites_trade_count_iso_df.rename(columns={'alpha_2':'Export_alpha_2',
                                            'alpha_3':'Export_alpha_3',
                                            'country_name':'Export_country_name',
                                           'lat':'expt_lat',
                                           'lon':'expt_lon'})

In [224]:
cites_trade_count_iso_df = cites_trade_count_iso_df.merge(country_code, how='left', left_on=['Importer'], right_on=['alpha_2'])
cites_trade_count_iso_df = cites_trade_count_iso_df.rename(columns={'alpha_2':'Import_alpha_2',
                                            'alpha_3':'Import_alpha_3', 
                                            'country_name':'Import_country_name',
                                            'lat':'impt_lat',
                                            'lon':'impt_lon'})

In [225]:
cites_trade_count_iso_expt = cites_trade_count_iso_df[cites_trade_count_iso_df['Type']=='Export']

### Trade of endangered species by country of export and import

In [226]:
endangered_fig = px.bar(cites_trade_count_iso_expt, 
                    x=cites_trade_count_iso_expt.Export_country_name, 
                    y=cites_trade_count_iso_expt.Coun_of_Taxon, 
                    color=cites_trade_count_iso_expt.Importer,
                    barmode='stack',
                    hover_data=['Export_country_name', 'Coun_of_Taxon', 'Quantity', 'Coun_of_Taxon', 'Import_country_name'], 
                    labels={
                        "Coun_of_Taxon":"Count of Species", 
                        "Export_country_name": "Export Country",
                        "Importer": "Import Country Code",
                        "Import_country_name": "Import Country", 
                        "Quantity": "Trade Quantity"
                        
                    },
                    title="Trade of Endangered Species by Country")
endangered_fig.show()

# Analytic Questions

## What is the most important trade in terms of volume? The least?

In [227]:
fig_most.show()

In [228]:
fig_least.show()

## What patterns exist between importing and exporting countries?

Observation: South and Central American Countries are predominantly exporting while Europe and Asian countries are importing. 

In [229]:
mapbox_fig.show()

## What country/countries trade the largest number of endangered species and what country do they export to?

In [123]:
endangered_fig.show()